# Sentiment Analysis

Autore: [Alessandro D'Orazio](https://alessandrodorazio.it) per [parliamodiai.it](https://parliamodiai.it)

In questo notebook, vengono mostrare delle tecniche per fare l'analisi del sentiment, cioè capire, partendo da un testo, se questo sia positivo o negativo. Utilizzeremo un dataset disponibile pubblicamente in cui vengono associate delle recensioni di Amazon ad un valore positivo nel caso in cui la recensione sia positiva, negativa altrimenti. <br>
Questo tipo di esercizio è detto di classificazione, poiché dati degli input (testi), essi devono essere inseriti in delle classi (sentiment positivo o negativo).

Verranno impiegati degli algoritmi base di AI, come:

- Regressione Logistica
- Support Vector Machine
- Random Forest Classifier

Successivamente, viene utilizzato l'[Ensemble Learning](https://it.wikipedia.org/wiki/Apprendimento_d%27insieme) per capire se possiamo trarre benefici da modelli più complessi.

In [1]:
# importazione delle dipendenze base
import pandas as pd
import numpy as np
from timeit import default_timer as timer

# Analisi del dataset

Il dataset contiene molte colonne per ogni recensione. Per esempio, sono presenti il nome del prodotto, la categoria, il marketplace in cui è stato effettuato l'acquisto (nazione) etc.

Essendo questo un esercizio di NLP, andremo ad utilizzare esclusivamente tre colonne: il titolo della recensione, il testo della recensione e il valore del sentiment.

In [2]:
df = pd.read_csv('product-reviews.csv', usecols=['review_headline', 'review_body', 'sentiment'])
df

,review_headline,review_body,sentiment
0,Five Stars,Great love it,1
1,Lots of ads Slow processing speed Occasionally...,Lots of ads<br />Slow processing speed<br />Oc...,0
2,Well thought out device,Excellent unit. The versatility of this table...,1
3,Not all apps/games we were looking forward to ...,I bought this on Amazon Prime so I ended up bu...,1
4,Five Stars,All Amazon products continue to meet my expect...,1
...,...,...,...
30841,A great upgrade for me from an older Kindle Fire!,[[VIDEOID:moP3B6GS5RL8LY]]I purchased the orig...,1
30842,Great Value for $139,I'm writing this review with the benefit of be...,1
30843,Even grandma has it figured out!,"I purchased this Kindle for my grandma, becaus...",1
30844,The Honda Accord of Tablets,I bought my tablet Fire HD 7 at Best Buy on th...,1


# Preprocessamento del testo

## Stop Words

Ogni recensione è scritta in linguaggio naturale, cioè il linguaggio che utilizziamo comunemente quando scriviamo qualcosa. Il linguaggio naturale contiene tante stop words, cioè parole comuni che non riguardano uno specifico argomento. Rientrano nelle stop words gli articoli, le preposizioni e le congiunzioni, poiché sono parole che possiamo ritrovare in ogni testo.

Considerando la seguente frase: "L'intelligenza artificiale è la capacità di un sistema artificiale di simulare l'intelligenza umana", possiamo considerare come stop words: `L'`, `la`, `di`, `un`. Questo significa che, rimuovendo le stop words, il testo diventa: 'intelligenza artificiale è capacità sistema artificiale simulare intelligenza umana'.

È importante rimuovere le stop words nell'ambito del NLP poiché esse potrebbero indurre i modelli in errore, per esempio, aumentando la possibilità per cui le recensioni contenenti la parola "La" vengano contrassegnate come positive o negative solo perché presente questo articolo.

In Python esiste un package chiamato `nltk` che fornisce delle utility nell'ambito del linguaggio naturale.

## Tokenizzazione

Un altro step molto importante è quello di tokenizzare i testi. In pratica, partendo da una stringa composta da 100 parole, attraverso la tokenizzazione la stringa viene convertita in un array di 100 elementi.

Dunque, riprendendo la frase "L'intelligenza artificiale è la capacità di un sistema artificiale di simulare l'intelligenza umana" il risultato sarà:
```
["L'intelligenza",
 'artificiale',
 'è',
 'la',
 'capacità',
 'di',
 'un',
 'sistema',
 'artificiale',
 'di',
 'simulare',
 "l'intelligenza",
 'umana']```



## Altre tecniche per il processamento

Esistono molte altre tecniche per il processamento dei testi, per esempio lo [stemming](https://it.wikipedia.org/wiki/Stemming) e la [lemmization](https://en.wikipedia.org/wiki/Lemmatization), che non utilizzeremo in questo notebook poiché saranno approfondite in altri notebook.

Una tecnica che utilizzeremo sarà invece quella di trasformare tutte le lettere in minuscolo, in modo tale da non avere differenza tra una parola scritta tutta in minuscolo ed una scritta tutta in maiuscolo (artificiale vs ARTIFICIALE).

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    if isinstance(text, str) is not True:
        return []
    stop_words = set(stopwords.words('english')) # il dataset è in inglese, dunque utilizziamo le stop words inglesi
    tokens = word_tokenize(text)
    processed_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token.isalpha()]
    return processed_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alessandrodorazio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alessandrodorazio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df['title_tokens'] = df['review_headline'].apply(preprocess_text)
df['body_tokens'] = df['review_body'].apply(preprocess_text)
print(df[['title_tokens', 'body_tokens']])

                                            title_tokens  \
0                                          [five, stars]   
1      [lots, ads, slow, processing, speed, occasiona...   
2                                [well, thought, device]   
3      [looking, forward, using, compatible, tablet, ...   
4                                          [five, stars]   
...                                                  ...   
30841              [great, upgrade, older, kindle, fire]   
30842                                     [great, value]   
30843                           [even, grandma, figured]   
30844                           [honda, accord, tablets]   
30845                                       [wo, regret]   

                                             body_tokens  
0                                          [great, love]  
1      [lots, ads, br, slow, processing, speed, br, o...  
2      [excellent, unit, versatility, tablet, besides...  
3      [bought, amazon, prime, ended, buyin

Dato che i modelli avranno solamente un testo in input, andiamo a concatenare il titolo e il corpo della recensione in un'unica colonna.

In [5]:
df['review_tokens'] = df['title_tokens'] + df['body_tokens']
df['review_tokens']

0                               [five, stars, great, love]
1        [lots, ads, slow, processing, speed, occasiona...
2        [well, thought, device, excellent, unit, versa...
3        [looking, forward, using, compatible, tablet, ...
4        [five, stars, amazon, products, continue, meet...
                               ...                        
30841    [great, upgrade, older, kindle, fire, videoid,...
30842    [great, value, writing, review, benefit, exper...
30843    [even, grandma, figured, purchased, kindle, gr...
30844    [honda, accord, tablets, bought, tablet, fire,...
30845    [wo, regret, impressive, piece, hardware, regr...
Name: review_tokens, Length: 30846, dtype: object

# Vettorizzazione

I modelli di intelligenza artificiale non sono in grado di lavorare con le parole, poiché necessitano di rappresentazioni matematiche. <br>
Per trasformare una parola in una rappresentazione matematica, vengono utilizzate delle tecniche di vettorizzazione.

## [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model)

Questa è la tecnica più semplice. In pratica, viene creato un vocabolario con tutte le parole utilizzate, e per ogni parola viene associato il numero di occorrenze.


## [TF-IDF](https://it.wikipedia.org/wiki/Tf-idf)

Attraverso TF-IDF (acronimo di Term Frequency-Inverse Document Frequency), viene calcolata l'importanza di una parola all'interno di un documento. <br>
Vengono calcolati due valori:

- Term Frequency (TF): la frequenza della parola nei documenti, cioè la percentuale di volte che viene utilizzata una determinata parola. Per esempio, se la parola "artificiale" è presente 5 volte nel documento 57, contenente 100 parole, allora TF(artificiale, 57)=5%. <br><br>

- Inverse Document Frequency (IDF): quanto è specifica una parola all'interno dei documenti. Viene computato per evitare che parole che compaiono molto spesso (es. stop words) non abbiano troppo peso rispetto a quelle più significative. Viene calcolato tramite il logaritmo del rapporto tra il numero di documenti totali e il numero di documenti in cui appare un dato termine. Dunque, se la parola artificiale compare in 12 documenti su 91, allora IDF(artificiale)=log(91/12)=0,87.

Dunque, il valore TF-IDF per ogni parola X nel documento Y, si calcola facendo TF(X, Y)*IDF(X). Più è alto il valore TF-IDF, più quella parola dovrebbe essere significativa.



## [Word2Vec](https://it.wikipedia.org/wiki/Word2vec)

Word2Vec è una rete neurale di word embedding (cioè vettorizzazione tramite deep learning) in cui ogni parola viene rappresentata come un punto in uno spazio multidimensionale. <br>
Il suo punto di forza è che date due parole correlate, queste saranno vicine nello spazio, dunque saranno considerate simili o in qualche modo correlate. <br>
Al contrario delle tecniche descritte precedentemente, Word2Vec è in grado di catturare anche il significato delle parole nel contesto (mentre Bag of Words e TF-IDF si limitano a contare le occorrenze delle parole).

[Paper ufficiale di Word2Vec](https://arxiv.org/pdf/1301.3781)

In questo notebook viene utilizzato Word2Vec tramite il package `gensim`, che fornisce delle utility pronte all'uso.

In [6]:
from gensim.models import Word2Vec
import gensim.downloader as api

# addestrare una rete Word2Vec da zero è time consuming, dunque carichiamo un modello già pronto
model = api.load('word2vec-google-news-300')

Dato che alcune parole presenti nelle recensioni non sono nella rete Word2Vec fornita da Gensim, rimuoveremo queste parole.<br>
In un ambito di produzione sarebbe opportuno invece riaddestrare la rete da zero. 

In [7]:
def document_vector(doc):
    doc = [word for word in doc if word in model]
    if len(doc) > 0:
        return np.mean(model[doc], axis=0)
    else:
        return np.zeros(model.vector_size,)

df['doc_vector'] = df['review_tokens'].apply(document_vector)
df['doc_vector']

0        [0.056793213, 0.036499023, 0.0335083, 0.146362...
1        [0.091103144, 0.02632795, -0.03199986, 0.06162...
2        [0.032694574, 0.04096225, -0.065703206, 0.0583...
3        [0.050921816, 0.016914137, -0.007219565, 0.107...
4        [-0.11839076, 0.11945452, 0.010864258, 0.14957...
                               ...                        
30841    [0.02131028, 0.03754676, -0.022550864, 0.04066...
30842    [0.050956424, 0.034219164, -0.02223143, 0.0548...
30843    [0.048681132, 0.022238566, -0.016763305, 0.093...
30844    [0.02453538, 0.062008925, -0.029453654, 0.0401...
30845    [0.060272217, 0.011199951, 0.027770996, 0.0010...
Name: doc_vector, Length: 30846, dtype: object

# Creazione dei dataset di training e test 

Andiamo ora a creare i dataset, in cui l'80% sarà per il training ed il restante 20% per il test.

In [8]:
from sklearn.model_selection import train_test_split

X = np.array(list(df['doc_vector']))
y = np.array(df['sentiment'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creazione e valutazione dei modelli

Come anticipato precedentemente, andremo ad utilizzare tre modelli differenti per analizzare quale ci fornirà il risultato più accurato.

## Metriche per la valutazione

Prima della valutazione dei modelli, è necessario parlare di alcune nozioni preliminari:
- True positive: predizioni corrette con output vero
- True negative: predizioni corrette con output falso
- False positive: predizioni errate con output vero
- False negative: predizioni errate con output falso

Andremo a valutare i modelli utilizzando le seguenti metriche.

#### Accuracy

L'accuracy rappresenta la percentuale di predizioni che il modello ha indovinato.

$$\text{Accuracy}=\frac {\text{Predizioni Corrette}} {\text{Predizioni totali}}$$

#### Precision

La precision rappresenta l'accuratezza delle predizioni positive.

$$\text{Precision}=\frac{\text{True positive}} {\text{True positive + False Positive}}$$


#### Recall

Il recall rappresenta il rapporto tra le predizioni vere corrette e le predizioni che dovevano essere contrassegnate come vere ma che sono state contrassegnate come false.

$$\text{Recall}=\frac{\text{True Positive}} {\text{True Positive + False Negative}}$$

#### F1-Score

L'F1-Score è una misura pesata tra precision e recall.<br>
Rappresenta quanto la precision e la recall siano bilanciate tra di loro. Un valore più elevato indica genericamente un modello complessivamente migliore.

$$
\
\text{F1-Score}=2 * \left( \frac {\text{Precision}*\text{Recall}} {\text{Precision}+\text{Recall}} \right)
\
$$

#### Support

Quanti elementi sono stati valutati.


## [Regressione Logistica](https://it.wikipedia.org/wiki/Modello_logit)

La regressione logistica è un modello statistico per modellare la probabilità di un risultato binario (vero/falso).<br>

All'atto pratico, nella regressione logistica, l'addestramento consiste nel determinare i pesi da applicare durante le predizioni, che avvengono tramite il calcolo di una combinazione lineare. I pesi vengono tipicamente stabiliti utilizzando un'estimator chiamato [Maximum Likelihood Estimation](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation#:~:text=In%20statistics%2C%20maximum%20likelihood%20estimation,observed%20data%20is%20most%20probable.)<br>
Successivamente, viene applicata una funzione sigmoide (detta anche logistica, da qui il nome), che porterà l'output della combinazione lineare in un range compreso tra 0 e 1. <br>
Infine, il valore ottenuto dalla funzione sigmoide servirà al modello per decidere se rispondere vero o falso. In pratica, viene stabilito un threshold, e se il valore è maggiore di questo threshold, allora il modello risponderà vero, altrimenti falso.

In [9]:
models_metrics = {}
results_metrics = [['model', 'accuracy', 'precision', 'recall', 'f1', 'elapsed']]
results_metrics

[['model', 'accuracy', 'precision', 'recall', 'f1', 'elapsed']]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

start = timer()
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train) # addestramento
y_pred = model.predict(X_test)

# valutazione del modello
elapsed_lr = (timer() - start)*1000
accuracy_lr = accuracy_score(y_test, y_pred)
precision_lr = precision_score(y_test, y_pred)
recall_lr = recall_score(y_test, y_pred)
f1_lr = f1_score(y_test, y_pred)

results_metrics.append(['Logistic Regression', accuracy_lr, precision_lr, recall_lr, f1_lr, elapsed_lr])

print("Logistic Regression Model Evaluation:")
print(f"Elapsed: {elapsed_lr:.2f}, Accuracy: {accuracy_lr:.2f}, Precision: {precision_lr:.2f}, Recall: {recall_lr:.2f}, F1-Score: {f1_lr:.2f}")
print("\nClassification Report for Logistic Regression:")
print(classification_report(y_test, y_pred))

Logistic Regression Model Evaluation:
Elapsed: 897.31, Accuracy: 0.90, Precision: 0.92, Recall: 0.96, F1-Score: 0.94

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.75      0.59      0.66      1018
           1       0.92      0.96      0.94      5152

    accuracy                           0.90      6170
   macro avg       0.83      0.78      0.80      6170
weighted avg       0.89      0.90      0.89      6170



## [Support Vector Machine](https://en.wikipedia.org/wiki/Support_vector_machine)

Le Support Vector Machine sono dei modelli di machine learning supervisionato che impiegano il concetto di [iperpiano](https://en.wikipedia.org/wiki/Hyperplane). <br>
Prendendo in considerazione un piano a due dimensioni, l'iperpiano è una linea di separazione. Viene impiegata in modo tale che i punti disposti da una parte della linea siano considerati come veri, mentre i punti dall'altra parte della linea come falsi. <br>
Il concetto di iperpiano si applica anche con piano con più dimensioni.

Una nozione fondamentale è quella del margine, cioè la distanza tra i punti più vicini che forniscono output differenti. Questi punti sono critici, poiché definiscono (o supportano) l'iperpiano e vengono chiamati support vector.

Forme di SVM:

- Lineari, cioè quelli in cui l'iperpiano generato è lineare nella sua accezione matematica
- Non lineari, cioè quando i dati non sono separabili linearmente, e dunque è necessario trasformare i dati in modo tale che arrivino ad una dimensione tale per cui sia possibile costruire un iperpiano lineare per separare i dati

Quando viene addestrata una SVM, il modello prova a trovare l'iperpiano corretto risolvendo un problema di ottimizzazione che massimizzi il margine.

Nell'esempio sarà utilizzato un kernel lineare, cioè in cui non avvengono trasformazioni rispetto allo spazio delle feature. Nel caso invece di un kernel polinomiale, i dati vengono trasformati in uno spazio polinomiale.

In [11]:
from sklearn.svm import SVC

start = timer()
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

elapsed_svm = (timer() - start)*1000
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

results_metrics.append(['Support Vector Machine', accuracy_svm, precision_svm, recall_svm, f1_svm, elapsed_svm])

print("SVM Model Evaluation:")
print(f"Elapsed: {elapsed_svm: .2f}, Accuracy: {accuracy_svm:.2f}, Precision: {precision_svm:.2f}, Recall: {recall_svm:.2f}, F1-Score: {f1_svm:.2f}")
print("\nClassification Report for SVM:")
print(classification_report(y_test, y_pred_svm))

SVM Model Evaluation:
Elapsed:  35421.15, Accuracy: 0.90, Precision: 0.93, Recall: 0.96, F1-Score: 0.94

Classification Report for SVM:
              precision    recall  f1-score   support

           0       0.74      0.61      0.67      1018
           1       0.93      0.96      0.94      5152

    accuracy                           0.90      6170
   macro avg       0.83      0.78      0.81      6170
weighted avg       0.90      0.90      0.90      6170



## [Random Forest](https://en.wikipedia.org/wiki/Random_forest)

Prima di parlare di Random Forest, bisogna parlare di [alberi di decisione](https://it.wikipedia.org/wiki/Albero_di_decisione). <br>
Un albero di decisione è un grafo aciclico in cui ogni conseguenza di una possibile decisione (es. valore X maggiore di 5) viene ramificata. Nel machine learning ogni nodo rappresenta una variabile (es. valore X) e i suoi figli rappresentano i valori possibili. <br>
Le foglie rappresentano il valore predetto. Per capire come è stato predetto il valore, è sufficiente attraversare l'albero dalla radice alla foglia.

I Random Forest sono un modello in cui vengono utilizzati molti alberi di decisione in modo tale da catturare più aspetti del dataset in ingresso. Nella classificazione, tipicamente il valore predetto è ciò che "votano" la maggior parte degli alberi, mentre nella regressione è la media tra i valori predetti.

In [12]:
from sklearn.ensemble import RandomForestClassifier

start = timer()
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 alberi nella foresta
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

elapsed_rf = (timer() - start)*1000
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

results_metrics.append(['Random Forest', accuracy_rf, precision_rf, recall_rf, f1_rf, elapsed_rf])

print("Random Forest Model Evaluation:")
print(f"Elapsed: {elapsed_rf: .2f}, Accuracy: {accuracy_rf:.2f}, Precision: {precision_rf:.2f}, Recall: {recall_rf:.2f}, F1-Score: {f1_rf:.2f}")
print("\nClassification Report for Random Forest:")
print(classification_report(y_test, y_pred_rf))

Random Forest Model Evaluation:
Elapsed:  30403.26, Accuracy: 0.88, Precision: 0.89, Recall: 0.98, F1-Score: 0.93

Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.78      0.38      0.51      1018
           1       0.89      0.98      0.93      5152

    accuracy                           0.88      6170
   macro avg       0.83      0.68      0.72      6170
weighted avg       0.87      0.88      0.86      6170



# Analisi dei risultati

In [13]:
pd.DataFrame(results_metrics[1:], columns=results_metrics[0]).round({'accuracy': 2, 'precision': 2, 'recall': 2, 'f1': 2, 'elapsed': 2})

,model,accuracy,precision,recall,f1,elapsed
0,Logistic Regression,0.90,0.92,0.96,0.94,897.31
1,Support Vector Machine,0.90,0.93,0.96,0.94,35421.15
2,Random Forest,0.88,0.89,0.98,0.93,30403.26


Come possiamo vedere, la regressione logistica e l'SVM hanno un accuracy ed una precision maggiore rispetto alle Random Forest.

Allo stesso tempo, il recall è migliore nelle Random Forest, poiché riesce a gestire meglio i casi di False Negative, cioè quei casi che dovrebbero essere veri ma che il modello predice come falsi.

L'F1-Score invece è tutto sommato simile, il che non ci porta ad avere una preferenza marcata.

La regressione logistica ha il minor costo computazionale, e per questo potrebbe essere considerato come il modello migliore in questa casistica. Se invece diamo molta importanza al non avere falsi negativi, le Random Forest sono quelle che performano meglio.

# [Ensemble learning](https://en.wikipedia.org/wiki/Ensemble_learning)

Passiamo ora all'Ensemble Learning, una modalità di apprendimento in cui vengono utilizzati più modelli sequenzialmente per aumentare l'accuratezza. <br> 
Anche le Random Forest sono considerate come esempio di Ensemble Learning, poiché partendo da alberi di decisione, ne vengono creati molti e utilizzati insieme per predire un valore. <br>

## [Bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating)

In particolare, le Random Forest utilizzano una tecnica chiamata **Bagging** (detto anche bootstrap aggregating), in cui vengono creati molti sottoinsiemi del dataset di partenza tramite il bootstrap sampling (ogni sottoinsieme è composto da elementi presi dal dataset e ogni elemento può comparire più di una volta). Successivamente, ogni modello viene addestrato su uno tra i sample creati. Infine, i risultati finali dipendono dalle predizioni di ogni modello. <br>
Nell'ambito della classificazione, solitamente vince la maggioranza, mentre nella regressione la media tra tutte le predizioni.


In questo notebook sono presenti tre tecniche per l'Ensemble Learning:
- AdaBoost
- Gradient Boosting
- Stacking

Prima di analizzare questi tre algoritmi, dobbiamo parlare di Gradient Descent.

## [Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent)

Partiamo dalla nozione di [gradient](https://en.wikipedia.org/wiki/Gradient): il gradient è un vettore che punta alla direzione del massimo tasso di aumento di una funzione scalare, e la sua grandezza corrisponde al tasso di aumento in quella direzione. <br>
Il Gradient Descent è un algoritmo di ottimizzazione per trovare dei coefficienti che minimizzino una funzione di costo il più possibile. L'idea alla base è di modificare i parametri iterativamente per minimizzare questa funzione di costo. La funzione di costo misura l'errore del modello. <br>
Nel Gradient Descent, il gradient è riferito alla funzione di costo, che fornisce la direzione per la salita più ripida. Muovendosi quindi in direzione opposta, è possibile ridurre l'errore. <br>
Più tecnicamente, si parte con dei valori iniziali per i parametri (coefficienti/pesi), e viene calcolato il gradient della funzione di costo rispetto ad ogni parametro. Successivamente, vengono modificati i parametri nella direzione che più riduce il costo. Vengono ripetuti questi step finché le modifiche tra un'iterazione e l'altra è molto piccola. È importante effettuare più iterazioni poiché il calcolo del gradient è basato sulla pendenza locale, quindi ci vogliono più iterazioni per raggiungere il minimo.

### Tipologie di Gradient Descent

- Batch Gradient Descent: computa il gradient della funzione di costo utilizzando l'intero dataset (computazionalmente costoso ma consistente)
- Stochastic Gradient Descent: utilizza un singolo sample alla volta, ma gli aggiornamenti potrebbero essere più "sporchi"
- Mini-batch Gradient Descent: computa il gradient e aggiorna i parametri utilizzando un sottoinsieme del dataset

# [Boosting](https://it.wikipedia.org/wiki/Boosting)

Il Boosting è una tecnica di Ensemble Learning che consiste nel combinare dei weak learners (algoritmi di learning semplici) in maniera sequenziale al fine di creare uno "strong learner", cioè un modello più accurato. <br>
Ogni weak learner, teoricamente, dovrebbe fornire dei risultati di poco migliori rispetto allo sparare a caso.  L'idea è che attraverso la loro combinazione venga prodotto un modello molto accurato. <br>
Ogni weak learner migliorerà le capacità del modello imparando dagli errori dei modelli precedenti, poiché si focalizza sui dati di addestramento che sono stato predetti male dai learner precedenti. <br>

# [AdaBoost](https://en.wikipedia.org/wiki/AdaBoost)

AdaBoost è un algoritmo di Boosting in cui i weak learners vengono combinati attraverso una somma pesata che rappresenta l'output finale. <br>
Dopo ogni iterazione, AdaBoost modifica i pesi delle istanze classificate in maniera errata in modo tale che i modelli successivi si possano focalizzare sulle casistiche più difficili. In pratica, prova a correggere gli errori aumentando il peso dei punti classificati erroneamente. I modelli con l'accuracy più alta influiranno maggiormente sulla predizione, computata come la somma pesata delle predizioni dei modelli weak.<br>
Utilizzeremo degli alberi di decisione come weak learners (che vengono impostati di default). <br>


In [14]:
from sklearn.ensemble import AdaBoostClassifier

start = timer()
ada_boost = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_boost.fit(X_train, y_train)
y_pred_ada = ada_boost.predict(X_test)

elapsed_ada = (timer() - start)*1000
accuracy_ada = accuracy_score(y_test, y_pred_ada)
precision_ada = precision_score(y_test, y_pred_ada)
recall_ada = recall_score(y_test, y_pred_ada)
f1_ada = f1_score(y_test, y_pred_ada)

results_metrics.append(['AdaBoost', accuracy_ada, precision_ada, recall_ada, f1_ada, elapsed_ada])

print("AdaBoost Model Evaluation:")
print(f"Elapsed: {elapsed_ada: .2f}, Accuracy: {accuracy_ada:.2f}, Precision: {precision_ada:.2f}, Recall: {recall_ada:.2f}, F1-Score: {f1_ada:.2f}")
print("\nClassification Report for AdaBoost:")
print(classification_report(y_test, y_pred_ada))

AdaBoost Model Evaluation:
Elapsed:  66402.29, Accuracy: 0.89, Precision: 0.92, Recall: 0.95, F1-Score: 0.93

Classification Report for AdaBoost:
              precision    recall  f1-score   support

           0       0.69      0.57      0.62      1018
           1       0.92      0.95      0.93      5152

    accuracy                           0.89      6170
   macro avg       0.80      0.76      0.78      6170
weighted avg       0.88      0.89      0.88      6170



# [Gradient Boosting](https://en.wikipedia.org/wiki/Gradient_boosting)

Nel Gradient Boosting, ogni albero di decisione viene costruito utilizzando l'errore residuale degli alberi precedenti. <br> 
Per errore residuale si intende la differenza tra il valore computato e il valore attuale.

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

start = timer()
gradient_boost = GradientBoostingClassifier(n_estimators=100, random_state=42)
gradient_boost.fit(X_train, y_train)
y_pred_gb = gradient_boost.predict(X_test)

elapsed_gb = (timer() - start)*1000
accuracy_gb = accuracy_score(y_test, y_pred_gb)
precision_gb = precision_score(y_test, y_pred_gb)
recall_gb = recall_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb)

results_metrics.append(['Gradient Boosting', accuracy_gb, precision_gb, recall_gb, f1_gb, elapsed_gb])

print("Gradient Boosting Model Evaluation:")
print(f"Elapsed: {elapsed_gb: .2f}, Accuracy: {accuracy_gb:.2f}, Precision: {precision_gb:.2f}, Recall: {recall_gb:.2f}, F1-Score: {f1_gb:.2f}")
print("\nClassification Report for Gradient Boosting:")
print(classification_report(y_test, y_pred_gb))

Gradient Boosting Model Evaluation:
Elapsed:  171424.11, Accuracy: 0.89, Precision: 0.91, Recall: 0.96, F1-Score: 0.94

Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.74      0.54      0.62      1018
           1       0.91      0.96      0.94      5152

    accuracy                           0.89      6170
   macro avg       0.83      0.75      0.78      6170
weighted avg       0.88      0.89      0.89      6170



# XGBoost

XGBoost è un'implementazione avanzata del Gradient Boosting, più veloce e tipicamente più performante. <br>
Include una [regolarizzazione](https://en.wikipedia.org/wiki/Regularization_(mathematics)) L1 ([Lasso](https://en.wikipedia.org/wiki/Lasso_(statistics))) e L2 ([Ridge](https://en.wikipedia.org/wiki/Ridge_regression)) che previene problematiche comuni come l'overfitting. <br>
Al contrario del Gradient Boosting, XGBoost sviluppa gli alberi di decisione fino alla loro massima profondità e successivamente li riduce per aumentare l'efficienza. <br>
Un'altra caratteristica di XGBoost è che effettua una cross validation ad ogni iterazione, rendendo più semplice il processo di decisione riguardo i round di boosting per prevenire l'overfitting.

In [16]:
import xgboost as xgb

start = timer()
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

elapsed_xgb = (timer() - start)*1000
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

results_metrics.append(['XGBoost', accuracy_xgb, precision_xgb, recall_xgb, f1_xgb, elapsed_xgb])

print("XGBoost Model Evaluation:")
print(f"Elapsed: {elapsed_xgb:.2f}, Accuracy: {accuracy_xgb:.2f}, Precision: {precision_xgb:.2f}, Recall: {recall_xgb:.2f}, F1-Score: {f1_xgb:.2f}")
print("\nClassification Report for XGBoost:")
print(classification_report(y_test, y_pred_xgb))

XGBoost Model Evaluation:
Elapsed: 1706.86, Accuracy: 0.90, Precision: 0.92, Recall: 0.96, F1-Score: 0.94

Classification Report for XGBoost:
              precision    recall  f1-score   support

           0       0.74      0.59      0.66      1018
           1       0.92      0.96      0.94      5152

    accuracy                           0.90      6170
   macro avg       0.83      0.78      0.80      6170
weighted avg       0.89      0.90      0.89      6170



# Stacking

Lo Stacking è una tecnica che combina molti modelli differenti in un unico modello, per esempio utilizzando una regressione logistica e una Random Forest. <br>
È composto da due livelli:
1. Modelli base: gli algoritmi che saranno eseguiti (es. regressione logistica, SVM etc)
2. Meta Learner: modello che viene addestrato basandosi sugli output dei modelli base per combinare al meglio i risultati

Dato che vengono utilizzati modelli base differenti, allora ci aspettiamo che possano essere catturati pattern di dati differenti e che quindi si possa raggiungere una maggior accuracy. <br>
Un problema dello stacking potrebbe essere l'[overfitting](https://it.wikipedia.org/wiki/Overfitting), cioè l'avere un modello troppo complesso (tanti parametri) rispetto al numero di dati da osservare. Sebbene tutti i modelli presenti in questo notebook possano soffrire di overfitting, lo Stacking è quello che può soffrirne di più, essendo il modello più complesso. <br>
Potrebbe quindi succedere che, tornando al problema originale, nel caso in cui sia presente una data parola in tutte le recensioni positive (es. la parola "stelle", come nelle recensioni in cui viene detto "cinque stelle"), allora il modello potrebbe fornire delle predizioni errate quando presente la suddetta parola.

In [17]:
from sklearn.ensemble import StackingClassifier

start = timer()
estimators = [
    ('lr', LogisticRegression()),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svr', SVC(kernel='linear', random_state=42))
]

stacking_classifier = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
stacking_classifier.fit(X_train, y_train)
y_pred_stack = stacking_classifier.predict(X_test)

elapsed_stack = (timer() - start)*1000
accuracy_stack = accuracy_score(y_test, y_pred_stack)
precision_stack = precision_score(y_test, y_pred_stack)
recall_stack = recall_score(y_test, y_pred_stack)
f1_stack = f1_score(y_test, y_pred_stack)

results_metrics.append(['Stacking', accuracy_stack, precision_stack, recall_stack, f1_stack, elapsed_stack])

print("Stacking Classifier Model Evaluation:")
print(f"Elapsed: {elapsed_stack: .2f}, Accuracy: {accuracy_stack:.2f}, Precision: {precision_stack:.2f}, Recall: {recall_stack:.2f}, F1-Score: {f1_stack:.2f}")
print("\nClassification Report for Stacking Classifier:")
print(classification_report(y_test, y_pred_stack))

Stacking Classifier Model Evaluation:
Elapsed:  290999.21, Accuracy: 0.90, Precision: 0.93, Recall: 0.96, F1-Score: 0.94

Classification Report for Stacking Classifier:
              precision    recall  f1-score   support

           0       0.74      0.63      0.68      1018
           1       0.93      0.96      0.94      5152

    accuracy                           0.90      6170
   macro avg       0.83      0.79      0.81      6170
weighted avg       0.90      0.90      0.90      6170



# Valutazione finale

In [18]:
metrics_df = pd.DataFrame(results_metrics[1:], columns=results_metrics[0])
metrics_df.round({'accuracy': 2, 'precision': 2, 'recall': 2, 'f1': 2, 'elapsed': 2})

,model,accuracy,precision,recall,f1,elapsed
0,Logistic Regression,0.90,0.92,0.96,0.94,897.31
1,Support Vector Machine,0.90,0.93,0.96,0.94,35421.15
2,Random Forest,0.88,0.89,0.98,0.93,30403.26
3,AdaBoost,0.89,0.92,0.95,0.93,66402.29
4,Gradient Boosting,0.89,0.91,0.96,0.94,171424.11
5,XGBoost,0.90,0.92,0.96,0.94,1706.86
6,Stacking,0.90,0.93,0.96,0.94,290999.21


Possiamo notare come il modello SVM e lo Stacking abbiano prodotto dei risultati praticamente uguali.

Se dobbiamo considerare solamente i modelli Ensemble, sicuramente lo Stacking è quello che performa meglio, ma è anche computazionalmente costoso. Se invece siamo disposti a sacrificare circa l'1% di precision, allora possiamo valutare il modello XGBoost o la regressione logistica che sono invece molto meno costosi (meno di 2 secondi vs 290). <br>
Stiamo in ogni caso parlando di valori molto vicini, a discapito invece di un costo computazionale molto differente. <br>

In conclusione, nel caso in cui l'aggiornamento del dataset di addestramento (e quindi del modello) avvenga molto poco frequentemente, si potrebbe valutare una SVM. Se invece il dataset cambia molto frequentemente, la scelta migliore potrebbe essere XGBoost o la regressione logistica